Define timer by \t 1000 //this means 1second call .z.ts once  
.z.ts must take one arguement, time

In [10]:
([]name:`symbol$();func:();time:`timestamp$())
([]name:`$();func:();time:"p"$())
show timer.job: flip `name`func`time!"s*p"$\:() //all three are the same, last will be shortest for many columns

name func time
--------------


name func time
--------------


name func time
--------------


In [11]:
meta timer.job //do note that the func will change type depending on first entry that goes in, we need make it generic
meta timer.job upsert (`;();.z.P) //adding a mixed list will make it well, can add identity too
meta timer.job upsert (`;::;.z.P)
1#flip `name`func`time!"s*p"$\:() //this is also a good way //adding empty row
timer.job upsert (`;`test;.z.P) //can add any rubbish in middle row now

c   | t f a
----| -----
name| s    
func|      
time| p    


c   | t f a
----| -----
name| s    
func|      
time| p    


c   | t f a
----| -----
name| s    
func|      
time| p    


name func time
--------------
              


name func time                         
---------------------------------------
     test 2020.04.02D22:27:15.136472000


#### upsert  
upsert[table;row]  
table: (symbol:in placed return symbol), (name: not in palce, return table)  
row/table: a list or a table,

#### xdesc
xdesc[col;table]  
col: must be symbol  
table: can be symbol or name

### Key to make them accept a table or a symbol
```
q)upsert
.[;();,;]
```
so to make drop works for both table and symbol we can define
```
q) newDrop:.[;();_;]
```


In [6]:
//however in this scenerio, we want our first row to be something that never runs// infinity time, so we won't see it
get `timer.job upsert (`;();0Wp) //back tick on table name, inplaced. return symbol. value to print it

name func time
--------------
          0W  


### Four Utilities function for timer
- merge: insert new timer events
- add: convert timestamps tp GMT before calling merge
- run: execute event from timer.job
- loop: called from kdb+ timer and check queue for event to run  
the kdb timer will be called every few millisecond.  
Note that kdb list/table is similar to python, appending and removing from the end is faster than from the front

In [28]:
system "l qtips/timer.q"
.timer.merge

k){$[#x;.Q.ord[>:;x]y;y]}[`time].[;();,;]


In [8]:
/ merge record(y) into table(x) in reverse chronological order
merge:`time xdesc upsert //upsert takes 2 input, table and data, so merge takes 2 input too

In [12]:
//.timer.add
{[t;n;f;tm] //table;eventName;function;time to run it
 r:(n;f;gtime tm);    //gtime converts to GMT, r is the data
 t:merge[t;$[0h>type tm;r;reverse flip r]]; //if atom, append atom, else flip will make any single element into right length
 t} //reverse is for first in, bottom of list
//Notice we do not hard code the table inside, now we have the flexibility of putting in a symbol or table
//symbol being a global table and later being a local table
//Note that global is more efficient as it is inplaced
//so this will return either a table or a symbol

{[t;n;f;tm]
 r:(n;f;gtime tm);
 t:merge[t;$[0h>type tm;r;reverse flip r]];
 t}


In [3]:
//.timer.loop  //periodically check timer.job for runnable event, to be used in .z.ts, last arguement is current time
{[t;tm] //normally we will not use while loop for kdb, but the table might change , while can inspect modified table
 while[tm>=last tms:t `time;t:run[t;-1+count tms;tm]]; 
 t} //note that: .z.ts:.timer.loop[`timer.job]

{[t;tm]
 while[tm>=last tms:t `time;t:run[t;-1+count tms;tm]];
 t}


In [2]:
//.timer.run 
{[t;i;tm] //table, row index, current time in GMT
 j:t i; //pick out i row
 t:.[t;();_;i]; //drop row i, note that t:t_i will not work with tables name, so need use apply method, so work on both
 r:value (f:j `func),ltime tm; //run the function with local time as input,
 if[not null r;merge[t;(j `name;f;tm+r)]]; //if above results is not null, repeat again 
 t}

{[t;i;tm]
 j:t i;
 t:.[t;();_;i];
 r:value (f:j `func),ltime tm;
 if[not null r;merge[t;(j `name;f;tm+r)]];
 t}


In [30]:
//.timer.until //delay, endTime, function, currentTime
{[d;et;f;tm]if[tm<et;@[value;f,tm;0N!];:d]} //if start time<end time, attempt to find value of f with arguement tm, else 0N!

{[d;et;f;tm]if[tm<et;@[value;f,tm;0N!];:d]}


In [36]:
//to run one off stuff just use add
e:(`.timer.until;0D00:00:05;.z.P+0D00:00:30;`.q.show); //every five seconds for next 30 sections show the time
.timer.add[`timer.job;`a;e;.z.P]

`timer.job


In [37]:
system "c 100 500"
timer.job 

name func                                                                        time                         
--------------------------------------------------------------------------------------------------------------
     ()                                                                          0W                           
a    (`.timer.until;0D00:00:05.000000000;2020.04.03D01:26:43.740606000;`.q.show) 2020.04.02D17:26:13.740616000
a    (`.timer.until;0D00:00:05.000000000;2020.04.03D01:26:07.351885000;`.q.show) 2020.04.02D17:25:37.352113000
